<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Fourier-based Option Pricing

For several reasons, it is beneficial to have available alternative valuation and pricing approaches to the Monte Carlo simulation approach. One application area is to **benchmark Monte Carlo-based valuation results** against other (potentially more accurate) results. Another area is **model calibration to liquidly traded vanilla instruments** where generally faster numerial methods can be applied.

This part introduces **Fouried-based valuation functions** and benchmarks valuation results from the "standard", simulation-based DX Analytics modeling approach to output of those functions. 

In [1]:
import dx
import datetime as dt

/home/yves/anaconda/lib/python2.7/site-packages/pandas/io/data.py:33: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


## Risk Factors

The examples and benchmarks to follow rely on four different models:

* geometric Brownian motion (Black-Scholes-Merton 1973)
* jump diffusion (Merton 1976)
* stochastic volatility (Heston 1993)
* stochastic volatility jump diffusion (Bates 1996)

For details on these models and the Fourier-based option pricing approach refer to Hilpisch (2015) (cf. http://eu.wiley.com/WileyCDA/WileyTitle/productCd-1119037999.html).

We first define the single **market and valuation environments**.

In [2]:
# constant short rate
r = dx.constant_short_rate('r', 0.01)

In [3]:
# geometric Brownian motion
me = dx.market_environment('me', dt.datetime(2015, 1, 1))
me.add_constant('initial_value', 100.)
me.add_constant('volatility', 0.2)
me.add_constant('final_date', dt.datetime(2015, 12, 31))
me.add_constant('currency', 'EUR')

In [4]:
# jump component
me.add_constant('lambda', 0.4)
me.add_constant('mu', -0.6)
me.add_constant('delta', 0.2)

In [5]:
# stochastic volatiltiy component
me.add_constant('rho', -.5)
me.add_constant('kappa', 5.0)
me.add_constant('theta', 0.02)
me.add_constant('vol_vol', 0.3)

In [6]:
# valuation environment
val_env = dx.market_environment('val_env', dt.datetime(2015, 1, 1))
val_env.add_constant('paths', 55000)
    # 25,000 paths
val_env.add_constant('frequency', 'D')
    # weekly frequency
val_env.add_curve('discount_curve', r)
val_env.add_constant('starting_date', dt.datetime(2015, 1, 1))
val_env.add_constant('final_date', dt.datetime(2015, 12, 31))

In [7]:
# add valuation environment to market environment
me.add_environment(val_env)

Equipped with the single market environments and the valuation environment, we can instantiate the **simulation model objects**.

In [8]:
gbm = dx.geometric_brownian_motion('gbm', me)

In [9]:
jd = dx.jump_diffusion('jd', me)

In [10]:
sv = dx.stochastic_volatility('sv', me)

In [11]:
svjd = dx.stoch_vol_jump_diffusion('svjd', me)

## Plain Vanilla Put and Call Options

Based on the just defined risk factors, we define 8 diffent options---a **European put and call option per risk factor**, respectively.

In [12]:
# market environment for the options
me_option = dx.market_environment('option', dt.datetime(2015, 1, 1))
me_option.add_constant('maturity', dt.datetime(2015, 12, 31))
me_option.add_constant('strike', 100.)
me_option.add_constant('currency', 'EUR')
me_option.add_environment(me)
me_option.add_environment(val_env)

In [13]:
euro_put_gbm = dx.valuation_mcs_european_single('euro_put', gbm, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_gbm = dx.valuation_mcs_european_single('euro_call', gbm, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [14]:
euro_put_jd = dx.valuation_mcs_european_single('euro_put', jd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_jd = dx.valuation_mcs_european_single('euro_call', jd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [15]:
euro_put_sv = dx.valuation_mcs_european_single('euro_put', sv, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_sv = dx.valuation_mcs_european_single('euro_call', sv, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

In [16]:
euro_put_svjd = dx.valuation_mcs_european_single('euro_put', svjd, me_option,
                                  'np.maximum(strike - maturity_value, 0)')
euro_call_svjd = dx.valuation_mcs_european_single('euro_call', svjd, me_option,
                                  'np.maximum(maturity_value - strike, 0)')

## Valuation Benchmarking

In this sub-section, we benchmark the **Monte Carlo value estimates** against the **Fourier-based pricing results**.

In [17]:
import numpy as np
import pandas as pd

We first define some parameters used throughout.

In [18]:
freq = '2m'  # used for maturity definitions
periods = 3  # number of intervals for maturity grid
strikes = 5  # number of strikes per maturity
initial_value = 100  # initial value for all risk factors
start = 0.8  # lowest strike in percent of spot
end = 1.2  # highest strike in percent of spot
start_date = '2015/3/1'  # start date for simulation/pricing

### Geometric Brownian Motion

We need to initialize the valuation object first.

In [19]:
euro_put_gbm.present_value()
  # method call needed for initialization

7.393567

There is a **valuation class for European put and call options in the Black-Scholes-Merton model** available called `BSM_european_option`. It is based on the analytical pricing formula for that model and is instantiated as follows:

In [20]:
bsm_option = dx.BSM_european_option('bsm_opt', me_option)

The following routine benchmarks the Monte Carlo value estimates for the **European put option** against the output from the valuation object based on the analytical pricing formula. The results are quite good since this model is quite easy to discretize exactly and therefore generally shows good convergence of the Monte Carlo estimates.

In [21]:
%%time
# European put
print '%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel')
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    bsm_option.maturity = maturity
    euro_put_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_put_gbm.update(strike=strike)
        mcs = euro_put_gbm.present_value()
        bsm_option.strike = strike
        ana = bsm_option.put_value()
        print '%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f ' \
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0334 |  0.0338 | -0.0004 |   -1.13 
0.244 |  90.000 |  0.6591 |  0.6524 |  0.0067 |    1.02 
0.244 | 100.000 |  3.7823 |  3.8130 | -0.0307 |   -0.81 
0.244 | 110.000 | 10.6224 | 10.6957 | -0.0733 |   -0.69 
0.244 | 120.000 | 19.7038 | 19.8537 | -0.1500 |   -0.76 
0.411 |  80.000 |  0.1816 |  0.1748 |  0.0069 |    3.94 
0.411 |  90.000 |  1.3171 |  1.3241 | -0.0070 |   -0.53 
0.411 | 100.000 |  4.8744 |  4.8985 | -0.0241 |   -0.49 
0.411 | 110.000 | 11.3483 | 11.4275 | -0.0792 |   -0.69 
0.411 | 120.000 | 19.9220 | 20.0325 | -0.1104 |   -0.55 
0.578 |  80.000 |  0.3847 |  0.3917 | -0.0071 |   -1.80 
0.578 |  90.000 |  1.9233 |  1.9466 | -0.0234 |   -1.20 
0.578 | 100.000 |  5.7235 |  5.7593 | -0.0358 |   -0.62 
0.578 | 110.000 | 12.0469 | 12.0934 | -0.0464 |   -0.38 
0.578 | 120.000 | 20.2342 | 20.3153 | -0.0811 |   -0.40 
CPU times: user 23.9 s, sys: 16.3 s, total: 40.1 s
Wall time: 40.3 s


The same now for the **European call option**.

In [22]:
euro_call_gbm.present_value()
  # method call needed for initialization

8.451126

In [23]:
%%time
# European calls
print '%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel')
for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
    euro_call_gbm.update(maturity=maturity)
    for strike in np.linspace(start, end, strikes) * initial_value:
        T = (maturity - me_option.pricing_date).days / 365.
        euro_call_gbm.update(strike=strike)
        mcs = euro_call_gbm.present_value()
        bsm_option.strike = strike
        bsm_option.maturity = maturity
        ana = bsm_option.call_value()
        print '%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.2f ' \
                % (T, strike, mcs, ana, mcs - ana, (mcs - ana) / ana * 100)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0778 | 20.2286 | -0.1508 |   -0.75 
0.244 |  90.000 | 10.7833 | 10.8716 | -0.0884 |   -0.81 
0.244 | 100.000 |  4.0361 |  4.0565 | -0.0204 |   -0.50 
0.244 | 110.000 |  0.9673 |  0.9636 |  0.0037 |    0.38 
0.244 | 120.000 |  0.1381 |  0.1460 | -0.0079 |   -5.40 
0.411 |  80.000 | 20.3868 | 20.5029 | -0.1161 |   -0.57 
0.411 |  90.000 | 11.6081 | 11.6932 | -0.0851 |   -0.73 
0.411 | 100.000 |  5.2743 |  5.3086 | -0.0343 |   -0.65 
0.411 | 110.000 |  1.8747 |  1.8787 | -0.0039 |   -0.21 
0.411 | 120.000 |  0.5213 |  0.5246 | -0.0033 |   -0.63 
0.578 |  80.000 | 20.7307 | 20.8528 | -0.1222 |   -0.59 
0.578 |  90.000 | 12.3800 | 12.4654 | -0.0854 |   -0.69 
0.578 | 100.000 |  6.2624 |  6.3357 | -0.0733 |   -1.16 
0.578 | 110.000 |  2.7073 |  2.7274 | -0.0201 |   -0.74 
0.578 | 120.000 |  1.0025 |  1.0070 | -0.0045 |   -0.45 
CPU times: user 24 s, sys: 10.7 s, total: 34.7 s
Wall time: 34.9 s


### Benchmarking Function

All other valuation benchmarks are generated with **Fourier-based pricing functions** for which the handling is identical. We therefore use the following function for the benchmarks from now on:

In [24]:
def valuation_benchmarking(valuation_object, fourier_function):
    print '%4s  | %7s | %7s | %7s | %7s | %7s' % ('T', 'strike', 'mcs', 'fou', 'dif', 'rel')
    for maturity in pd.date_range(start=start_date, freq=freq, periods=periods):
        valuation_object.update(maturity=maturity)
        me_option.add_constant('maturity', maturity)
        for strike in np.linspace(start, end, strikes) * initial_value:
            T = (maturity - me_option.pricing_date).days / 365.
            valuation_object.update(strike=strike)
            mcs = valuation_object.present_value()
            me_option.add_constant('strike', strike)
            fou = fourier_function(me_option)
            print '%4.3f | %7.3f | %7.4f | %7.4f | %7.4f | %7.3f ' \
                % (T, strike, mcs, fou, mcs - fou, (mcs - fou) / fou * 100)

### Jump Diffusion

The next model is the jump diffusion as proposed by **Merton (1976)**.

In [25]:
euro_put_jd.present_value()
  # method call needed for initialization

13.974479

There is a Fourier-based pricing function available which is called `M76_put_value` and which is used for the benchmarking for the **European put options** that follows.

In [26]:
%time valuation_benchmarking(euro_put_jd, dx.M76_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1443 |  2.1594 | -0.0152 |  -0.702 
0.244 |  90.000 |  3.2136 |  3.2826 | -0.0691 |  -2.104 
0.244 | 100.000 |  5.8272 |  5.8842 | -0.0570 |  -0.968 
0.244 | 110.000 | 11.5222 | 11.6115 | -0.0893 |  -0.769 
0.244 | 120.000 | 19.8792 | 20.0857 | -0.2065 |  -1.028 
0.411 |  80.000 |  3.4601 |  3.4505 |  0.0095 |   0.276 
0.411 |  90.000 |  5.2433 |  5.2162 |  0.0271 |   0.520 
0.411 | 100.000 |  8.2471 |  8.2266 |  0.0205 |   0.249 
0.411 | 110.000 | 13.3642 | 13.4430 | -0.0788 |  -0.586 
0.411 | 120.000 | 20.7725 | 20.9238 | -0.1513 |  -0.723 
0.578 |  80.000 |  4.5918 |  4.6090 | -0.0172 |  -0.373 
0.578 |  90.000 |  6.9404 |  6.8782 |  0.0622 |   0.904 
0.578 | 100.000 | 10.1243 | 10.2077 | -0.0833 |  -0.817 
0.578 | 110.000 | 15.0870 | 15.2251 | -0.1381 |  -0.907 
0.578 | 120.000 | 22.0400 | 22.0886 | -0.0486 |  -0.220 
CPU times: user 1min 3s, sys: 37.9 s, total: 1min 41s
Wall time: 1min 41s


Accordingly, the benchmarking for the **European call options** based on the Fourier-based `M76_call_value` function.

In [27]:
euro_call_jd.present_value()
  # method call needed for initialization

15.157772

In [28]:
%time valuation_benchmarking(euro_call_jd, dx.M76_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1773 | 22.3543 | -0.1770 |  -0.792 
0.244 |  90.000 | 13.3838 | 13.5018 | -0.1180 |  -0.874 
0.244 | 100.000 |  6.0844 |  6.1277 | -0.0433 |  -0.707 
0.244 | 110.000 |  1.8713 |  1.8794 | -0.0081 |  -0.430 
0.244 | 120.000 |  0.3850 |  0.3780 |  0.0070 |   1.863 
0.411 |  80.000 | 23.6591 | 23.7786 | -0.1195 |  -0.503 
0.411 |  90.000 | 15.4640 | 15.5853 | -0.1214 |  -0.779 
0.411 | 100.000 |  8.6566 |  8.6367 |  0.0199 |   0.230 
0.411 | 110.000 |  3.8999 |  3.8941 |  0.0058 |   0.150 
0.411 | 120.000 |  1.4253 |  1.4160 |  0.0093 |   0.658 
0.578 |  80.000 | 24.9173 | 25.0701 | -0.1528 |  -0.610 
0.578 |  90.000 | 17.3213 | 17.3970 | -0.0756 |  -0.435 
0.578 | 100.000 | 10.7039 | 10.7841 | -0.0802 |  -0.744 
0.578 | 110.000 |  5.8432 |  5.8591 | -0.0159 |  -0.271 
0.578 | 120.000 |  2.7895 |  2.7803 |  0.0092 |   0.332 
CPU times: user 59.2 s, sys: 17 s, total: 1min 16s
Wall time: 1min 16s


### Stochastic Volatility

Stochastic volatility models like the one of **Heston (1993)** are popular to reproduce implied volatility smiles observed in markets. First, the benchmarking for the **European put options** using the Fourier-based `H93_put_value` function.

In [29]:
euro_put_sv.present_value()
  # method call needed for initialization

5.273366

In [30]:
%time valuation_benchmarking(euro_put_sv, dx.H93_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  0.0471 |  0.0504 | -0.0033 |  -6.546 
0.244 |  90.000 |  0.5506 |  0.5741 | -0.0235 |  -4.096 
0.244 | 100.000 |  3.2441 |  3.3204 | -0.0763 |  -2.297 
0.244 | 110.000 | 10.1365 | 10.2404 | -0.1039 |  -1.014 
0.244 | 120.000 | 19.5859 | 19.7354 | -0.1494 |  -0.757 
0.411 |  80.000 |  0.1497 |  0.1603 | -0.0106 |  -6.638 
0.411 |  90.000 |  0.9461 |  1.0063 | -0.0602 |  -5.980 
0.411 | 100.000 |  3.9028 |  4.0234 | -0.1206 |  -2.996 
0.411 | 110.000 | 10.4425 | 10.5487 | -0.1062 |  -1.007 
0.411 | 120.000 | 19.4991 | 19.6293 | -0.1301 |  -0.663 
0.578 |  80.000 |  0.2528 |  0.2838 | -0.0310 | -10.938 
0.578 |  90.000 |  1.2892 |  1.3631 | -0.0739 |  -5.420 
0.578 | 100.000 |  4.3867 |  4.5467 | -0.1600 |  -3.518 
0.578 | 110.000 | 10.7024 | 10.8389 | -0.1365 |  -1.259 
0.578 | 120.000 | 19.4523 | 19.5767 | -0.1244 |  -0.635 
CPU times: user 50.8 s, sys: 27.8 s, total: 1min 18s
Wall time: 1min 19s


Second, the benchmarking for the **European call options** based on the Fourier-based `H93_call_value` function.

In [31]:
euro_call_sv.present_value()
  # method call needed for initialization

6.271918

In [32]:
%time valuation_benchmarking(euro_call_sv, dx.H93_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 20.0897 | 20.2453 | -0.1556 |  -0.768 
0.244 |  90.000 | 10.6936 | 10.7933 | -0.0997 |  -0.924 
0.244 | 100.000 |  3.5096 |  3.5639 | -0.0543 |  -1.524 
0.244 | 110.000 |  0.4850 |  0.5083 | -0.0233 |  -4.592 
0.244 | 120.000 |  0.0239 |  0.0276 | -0.0037 | -13.314 
0.411 |  80.000 | 20.3638 | 20.4884 | -0.1246 |  -0.608 
0.411 |  90.000 | 11.2711 | 11.3754 | -0.1042 |  -0.916 
0.411 | 100.000 |  4.3215 |  4.4335 | -0.1120 |  -2.527 
0.411 | 110.000 |  0.9215 |  0.9998 | -0.0784 |  -7.838 
0.411 | 120.000 |  0.1069 |  0.1214 | -0.0145 | -11.966 
0.578 |  80.000 | 20.6397 | 20.7450 | -0.1053 |  -0.507 
0.578 |  90.000 | 11.7429 | 11.8818 | -0.1389 |  -1.169 
0.578 | 100.000 |  4.9382 |  5.1231 | -0.1849 |  -3.609 
0.578 | 110.000 |  1.3668 |  1.4729 | -0.1061 |  -7.202 
0.578 | 120.000 |  0.2398 |  0.2684 | -0.0286 | -10.663 
CPU times: user 50.6 s, sys: 24 s, total: 1min 14s
Wall time: 1min 14s


### Stochastic Volatility Jump-Diffusion

Finally, we consider the combination of the stochastic volatility and jump diffusion models from before as proposed by **Bates (1996)**. The Fourier-based pricing function for **European put options** is called `B96_put_value`.

In [33]:
euro_put_svjd.present_value()
  # method call needed for initialization

12.867605

In [34]:
%time valuation_benchmarking(euro_put_svjd, dx.B96_put_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 |  2.1443 |  2.1638 | -0.0195 |  -0.902 
0.244 |  90.000 |  3.1708 |  3.2761 | -0.1053 |  -3.215 
0.244 | 100.000 |  5.4637 |  5.5889 | -0.1252 |  -2.240 
0.244 | 110.000 | 10.9612 | 11.0733 | -0.1121 |  -1.012 
0.244 | 120.000 | 19.6104 | 19.8344 | -0.2239 |  -1.129 
0.411 |  80.000 |  3.3386 |  3.4366 | -0.0980 |  -2.851 
0.411 |  90.000 |  5.0517 |  5.1339 | -0.0822 |  -1.602 
0.411 | 100.000 |  7.6055 |  7.7747 | -0.1691 |  -2.175 
0.411 | 110.000 | 12.3378 | 12.5744 | -0.2366 |  -1.882 
0.411 | 120.000 | 20.0617 | 20.1827 | -0.1210 |  -0.600 
0.578 |  80.000 |  4.3389 |  4.5480 | -0.2091 |  -4.598 
0.578 |  90.000 |  6.4655 |  6.7163 | -0.2508 |  -3.734 
0.578 | 100.000 |  9.3873 |  9.6585 | -0.2711 |  -2.807 
0.578 | 110.000 | 14.0120 | 14.1776 | -0.1657 |  -1.168 
0.578 | 120.000 | 20.7395 | 20.9423 | -0.2028 |  -0.968 
CPU times: user 1min 27s, sys: 29.1 s, total: 1min 56s
Wall time: 1min 56s


The Fourier-based counterpart function for **European call options** is called `B96_call_value`.

In [35]:
euro_call_svjd.present_value()
  # method call needed for initialization

13.908154

In [36]:
%time valuation_benchmarking(euro_call_svjd, dx.B96_call_value)

   T  |  strike |     mcs |     fou |     dif |     rel
0.244 |  80.000 | 22.1113 | 22.3587 | -0.2474 |  -1.107 
0.244 |  90.000 | 13.3566 | 13.4953 | -0.1387 |  -1.027 
0.244 | 100.000 |  5.7408 |  5.8325 | -0.0917 |  -1.572 
0.244 | 110.000 |  1.2754 |  1.3411 | -0.0658 |  -4.906 
0.244 | 120.000 |  0.1129 |  0.1266 | -0.0137 | -10.806 
0.411 |  80.000 | 23.5712 | 23.7647 | -0.1934 |  -0.814 
0.411 |  90.000 | 15.2728 | 15.5030 | -0.2302 |  -1.485 
0.411 | 100.000 |  7.9588 |  8.1848 | -0.2260 |  -2.761 
0.411 | 110.000 |  2.8724 |  3.0255 | -0.1531 |  -5.059 
0.411 | 120.000 |  0.6251 |  0.6749 | -0.0498 |  -7.375 
0.578 |  80.000 | 24.6623 | 25.0091 | -0.3469 |  -1.387 
0.578 |  90.000 | 16.9346 | 17.2351 | -0.3005 |  -1.743 
0.578 | 100.000 |  9.9695 | 10.2349 | -0.2654 |  -2.593 
0.578 | 110.000 |  4.5932 |  4.8117 | -0.2185 |  -4.541 
0.578 | 120.000 |  1.5200 |  1.6340 | -0.1140 |  -6.974 
CPU times: user 1min 27s, sys: 28.4 s, total: 1min 55s
Wall time: 1min 55s


## Sources of Errors

Numerical methods like Monte Carlo simulation might suffer from different **sources of errors**, like for example:

* **discretization error**: every **discretization of a continuous time interval**---or a continuous state space to this end---leads to a so-called discretization error
* **approximation errors**: DX Analytics uses in several places approximative, **Euler-based discretization schemes** (e.g. for performance reasons and to allow for consistent correlation modeling) which are known to be biased
* **numerical errors**: the approximation of a continuous probability distribution by a **finite, discrete set of (pseudo-) random numbers** introduces also errors

**Copyright, License & Disclaimer**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

DX Analytics (the "dx library") is licensed under the GNU Affero General Public License
version 3 or later (see [http://www.gnu.org/licenses/](http://www.gnu.org/licenses/)).

DX Analytics comes with no representations
or warranties, to the extent permitted by applicable law.


<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

[http://tpq.io](http://tpq.io) | [team@tpq.io](mailto:team@tpq.io) | [http://twitter.com/dyjh](http://twitter.com/dyjh)

**Quant Platform** |
[http://quant-platform.com](http://quant-platform.com)

**Derivatives Analytics with Python (Wiley Finance)** |
[http://derivatives-analytics-with-python.com](http://derivatives-analytics-with-python.com)

**Python for Finance (O'Reilly)** |
[http://python-for-finance.com](http://python-for-finance.com)